In [1]:
!git clone https://github.com/UniversalDependencies/UD_Vietnamese-VTB.git

Cloning into 'UD_Vietnamese-VTB'...


remote: Enumerating objects: 802, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 802 (delta 32), reused 27 (delta 23), pack-reused 763 (from 3)
Receiving objects: 100% (802/802), 20.76 MiB | 14.19 MiB/s, done.
Resolving deltas: 100% (431/431), done.


In [ ]:
!pip install conllu
!pip install torch
!pip install numpy
!pip install gensim
!pip install matplotlib

In [2]:
!pip install transformers

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from conllu import parse_incr
from collections import Counter
import numpy as np
import random
import os
import copy
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer


In [2]:
import torch
import numpy as np
import random
import os

def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Nếu dùng multi-GPU
    
    # Đảm bảo tính toán trên cuDNN là deterministic (có thể làm chậm tốc độ train một chút)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"✅ Đã thiết lập random seed: {seed}")

set_seed(42)

✅ Đã thiết lập random seed: 42


In [3]:
def map_sentence_to_ids(sentences, p_vocab, d_vocab):
    """Chuyển đổi toàn bộ nhãn UPOS và DEPREL trong câu sang ID trước khi training."""
    for sent in sentences:
        for token in sent:
            token['upos_id'] = p_vocab.get(token['upos'], p_vocab.get("<UNK>", 0))
            token['deprel_id'] = d_vocab.get(token['deprel'], 0)
    return sentences

In [35]:
import random
import copy

class Oracle:
    def __init__(self, vocab_pos, vocab_deps):
        self.p_vocab = vocab_pos
        self.d_vocab = vocab_deps
        # Danh sách ID các nhãn quan hệ để giả lập việc "gán nhầm"
        self.all_dep_ids = [v for k, v in vocab_deps.items() if k not in ["<NULL>", "<ROOT>", "<UNK>"]]
        self.action_map = self._build_action_map()

    def _build_action_map(self):
        mapping = {"SHIFT": 0}
        idx = 1
        for dep in self.d_vocab:
            if dep not in ["<NULL>", "<ROOT>", "<UNK>"]:
                mapping[f"LEFT-ARC_{dep}"] = idx
                idx += 1
                mapping[f"RIGHT-ARC_{dep}"] = idx
                idx += 1
        return mapping

    def extract_features(self, stack, buffer, sentence, sent_idx, left_children_map, right_children_map, 
                         stack_size=3, buffer_size=3, mutate=False):
        # 1. STACK VIEW
        s_window = stack[-stack_size:] if stack else [0]
        if len(s_window) < stack_size:
            # Pad LEFT để các phần tử quan trọng (s0, s1) nằm ở cuối list giống như stack thật
            s_window = [None] * (stack_size - len(s_window)) + s_window 
            
        s_feats = {
            'words': [], 'pos': [],
            'l_child_idx': [], 'l_child_dep': [],  'l_child_pos': [],
            'r_child_idx': [], 'r_child_dep': [],  'r_child_pos': [] 
        }

        for i, idx in enumerate(s_window):
            if idx is None:
                for k in s_feats: s_feats[k].append(0)
                continue

            s_feats['words'].append(idx)
            if idx == 0: # ROOT
                s_feats['pos'].append(0)
                for k in ['l_child_idx', 'l_child_dep', 'l_child_pos', 'r_child_idx', 'r_child_dep', 'r_child_pos']:
                    s_feats[k].append(0)
            else:
                token = sentence[idx - 1]
                s_feats['pos'].append(token['upos_id'])

                l_children = left_children_map.get(idx, [])
                r_children = right_children_map.get(idx, [])

                # Lấy thông tin con thực tế (Gold)
                l_idx, l_dep, l_pos = l_children[0] if l_children else (0, 0, 0)
                r_idx, r_dep, r_pos = r_children[-1] if r_children else (0, 0, 0)

                # --- LOGIC MUTATION (GIẢ LẬP SAI BƯỚC TRƯỚC) ---
                # i < stack_size - 1 đảm bảo chỉ mutate s1, s2... (các từ đã nằm lâu trên stack)
                if mutate and i < (stack_size - 1):
                    # Giả lập việc gán nhầm nhãn quan hệ cho các con của s1, s2
                    if l_dep != 0 and random.random() < 0.5:
                        l_dep = random.choice(self.all_dep_ids)
                    if r_dep != 0 and random.random() < 0.5:
                        r_dep = random.choice(self.all_dep_ids)
                # -----------------------------------------------

                s_feats['l_child_idx'].append(l_idx)
                s_feats['l_child_dep'].append(l_dep)
                s_feats['l_child_pos'].append(l_pos)
                s_feats['r_child_idx'].append(r_idx)
                s_feats['r_child_dep'].append(r_dep)
                s_feats['r_child_pos'].append(r_pos)

        # 2. BUFFER VIEW
        b_window = buffer[:buffer_size] if buffer else [0]
        if len(b_window) < buffer_size:
            b_window = b_window + [None] * (buffer_size - len(b_window))
            
        b_feats = {'words': [], 'pos': []}
        for idx in b_window:
            if idx is None:
                b_feats['words'].append(0); b_feats['pos'].append(0)
                continue
            b_feats['words'].append(idx)
            b_feats['pos'].append(0 if idx == 0 else sentence[idx-1]['upos_id'])

        return {'stack': s_feats, 'buffer': b_feats, 'sent_id': sent_idx}

    def create_training_data_multiview(self, sentence, sent_idx, aug_p=0.2):
        """
        aug_p: Xác suất tạo thêm một mẫu dữ liệu nhiễu tại mỗi bước transition.
        """
        stack, buffer = [0], list(range(1, len(sentence) + 1))
        left_children, right_children = {}, {}
        train_examples = []
        gold_heads = {token['id']: token['head'] for token in sentence}

        while buffer or len(stack) > 1:
            # 1. Trích xuất đặc trưng GOLD (Dữ liệu sạch)
            feat_gold = self.extract_features(stack, buffer, sentence, sent_idx, left_children, right_children, mutate=False)

            # --- Xác định Action ID đúng ---
            target_action_id = None
            if len(stack) >= 2:
                s1, s2 = stack[-1], stack[-2]
                if gold_heads.get(s2) == s1: # Left-Arc
                    token_s2 = sentence[s2 - 1]
                    target_action_id = self.action_map.get(f"LEFT-ARC_{token_s2['deprel']}", 0)
                elif gold_heads.get(s1) == s2 and not any(gold_heads.get(b) == s1 for b in buffer): # Right-Arc
                    token_s1 = sentence[s1 - 1]
                    target_action_id = self.action_map.get(f"RIGHT-ARC_{token_s1['deprel']}", 0)
                elif buffer: # Shift
                    target_action_id = self.action_map["SHIFT"]
            elif buffer:
                target_action_id = self.action_map["SHIFT"]

            if target_action_id is None: break

            # 2. THÊM DỮ LIỆU VÀO TẬP HUẤN LUYỆN
            # Thêm mẫu Gold (luôn giữ nguyên dữ liệu gốc)
            train_examples.append((feat_gold, target_action_id))

            # Augmentation: Thêm mẫu bị mutation (giả lập sai bước trước)
            if aug_p > 0 and random.random() < aug_p:
                feat_mutated = self.extract_features(stack, buffer, sentence, sent_idx, 
                                                     left_children, right_children, mutate=True)
                # Dạy mô hình: "Dù đặc trưng này bị nhiễu (mutation), hành động đúng vẫn là target_action_id"
                train_examples.append((feat_mutated, target_action_id))

            # 3. CẬP NHẬT TRẠNG THÁI (Sử dụng Gold Action để đi tiếp - Teacher Forcing)
            if target_action_id == self.action_map["SHIFT"]:
                stack.append(buffer.pop(0))
            else:
                action_name = [k for k, v in self.action_map.items() if v == target_action_id][0]
                if action_name.startswith("LEFT-ARC"):
                    child = stack.pop(-2)
                    head = stack[-1]
                    if head not in left_children: left_children[head] = []
                    left_children[head].append((child, sentence[child-1]['deprel_id'], sentence[child-1]['upos_id']))
                else: # RIGHT-ARC
                    child = stack.pop(-1)
                    head = stack[-1]
                    if head not in right_children: right_children[head] = []
                    right_children[head].append((child, sentence[child-1]['deprel_id'], sentence[child-1]['upos_id']))

        return train_examples

In [36]:
def load_data_and_build_vocab(file_path, is_train=True, word_v=None, pos_v=None, dep_v=None):
    sentences = []
    word_counts = Counter()
    pos_counts = Counter()
    dep_counts = Counter()

    for path in file_path:

        with open(path, "r", encoding="utf-8") as f:
          for tokenlist in parse_incr(f):
              # Chỉ thêm những câu có nội dung
              if len(tokenlist) > 0:
                  if not is_train or 'train' in path:
                      sentences.append(tokenlist)
                  if is_train:
                      for token in tokenlist:
                          word_counts[token['form'].lower()] += 1
                          pos_counts[token['upos']] += 1
                          dep_counts[token['deprel']] += 1

    if is_train:
        # Khởi tạo ID theo yêu cầu: NULL=0, ROOT=1, UNK=2
        def create_mapping(counts, is_dep=False):
            vocab = {"<NULL>": 0, "<ROOT>": 1}
            if not is_dep: vocab["<UNK>"] = 2
            for item, count in counts.items():
                if item not in vocab and (is_dep or count > 1):
                    vocab[item] = len(vocab)
            return vocab

        word_v = create_mapping(word_counts)
        pos_v = create_mapping(pos_counts)
        dep_v = create_mapping(dep_counts, is_dep=True)
        return sentences, word_v, pos_v, dep_v

    return sentences

In [37]:
def preprocess_data(train_path, dev_path, test_path):
    # BƯỚC 1: Load dữ liệu và chỉ xây dựng Vocab cho POS và Dep
    # Bạn có thể dùng lại hàm load_data_and_build_vocab cũ nhưng bỏ qua phần word_v
    print("Đang đọc tập Train và xây dựng Vocab cho POS/Dep...")
    train_sentences, _, p_vocab, d_vocab = load_data_and_build_vocab([train_path], is_train=True)

    # BƯỚC 2: Khởi tạo Oracle mới (Không dùng w_vocab)
    # Đảm bảo bạn đã cập nhật lớp Oracle như hướng dẫn ở bước trước
    oracle = Oracle(p_vocab, d_vocab)

    # BƯỚC 3: Tạo dữ liệu huấn luyện (X, y) dạng Sequence
    all_training_data = []
    print(f"Đang tạo chuỗi transitions cho {len(train_sentences)} câu tập Train...")

    for idx, sentence in enumerate(train_sentences):
        try:
            # create_training_data bây giờ trả về danh sách các dict chứa chuỗi indices
            examples = oracle.create_training_data(sentence, idx)
            all_training_data.extend(examples)
        except Exception as e:
            print(f"Lỗi logic tại câu: {sentence.metadata['text']} - {e}")
            continue

    print(f"✅ Thành công! Tổng số bước transition huấn luyện: {len(all_training_data)}")

    # BƯỚC 4: Load tập Dev và Test (giữ nguyên câu gốc để PhoBERT encode sau này)
    print("Đang chuẩn bị tập Dev và Test...")
    dev_sentences = load_data_and_build_vocab([dev_path], is_train=False)
    test_sentences = load_data_and_build_vocab([test_path], is_train=False)

    print(f"Số câu: Train={len(train_sentences)}, Dev={len(dev_sentences)}, Test={len(test_sentences)}")

    # Trả về thêm train_sentences để tí nữa dùng lấy PhoBERT embedding
    return all_training_data, (p_vocab, d_vocab), (train_sentences, dev_sentences, test_sentences)

In [38]:
import torch
from torch.utils.data import Dataset

class TransitionDataset(Dataset):
    def __init__(self, training_examples):
        self.examples = training_examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        feat, action_id = self.examples[idx]

        # Stack features
        s_words = torch.tensor(feat['stack']['words'], dtype=torch.long)
        s_pos = torch.tensor(feat['stack']['pos'], dtype=torch.long)
        s_lc_idx = torch.tensor(feat['stack']['l_child_idx'], dtype=torch.long)
        s_lc_dep = torch.tensor(feat['stack']['l_child_dep'], dtype=torch.long) # MỚI
        s_lc_pos = torch.tensor(feat['stack']['l_child_pos'], dtype=torch.long) # MỚI
        s_rc_idx = torch.tensor(feat['stack']['r_child_idx'], dtype=torch.long)
        s_rc_dep = torch.tensor(feat['stack']['r_child_dep'], dtype=torch.long) # MỚI
        s_rc_pos = torch.tensor(feat['stack']['r_child_pos'], dtype=torch.long) # MỚI

        # Buffer features
        b_words = torch.tensor(feat['buffer']['words'], dtype=torch.long)
        b_pos = torch.tensor(feat['buffer']['pos'], dtype=torch.long)

        return (s_words, s_pos, s_lc_idx, s_lc_dep, s_lc_pos, s_rc_idx, s_rc_dep, s_rc_pos,
                b_words, b_pos, torch.tensor(action_id), feat['sent_id'])

In [39]:
from torch.nn.utils.rnn import pad_sequence
def multiview_collate_fn(batch):
    # Unpack 11 thành phần
    (s_words, s_pos, s_lc_idx, s_lc_dep, s_lc_pos, s_rc_idx, s_rc_dep, s_rc_pos,
     b_words, b_pos, labels, sent_ids) = zip(*batch)

    # Helper function để pad cho gọn
    def pad(tensors): return pad_sequence(tensors, batch_first=True, padding_value=0)

    return (
        pad(s_words), pad(s_pos),
        pad(s_lc_idx), pad(s_lc_dep), pad(s_lc_pos), # Pad LC
        pad(s_rc_idx), pad(s_rc_dep), pad(s_rc_pos), # Pad RC
        pad(b_words), pad(b_pos),
        torch.stack(labels), sent_ids
    )

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SubViewProjector(nn.Module):
    """Chiếu các nhóm đặc trưng (Word+POS/Dep) về cùng một không gian ẩn."""
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.Mish(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim),
            nn.LayerNorm(output_dim),
            nn.Mish(),
            nn.Dropout(dropout_rate)
        )
    def forward(self, x):
        return self.net(x)
        
class GatedFusion(nn.Module):
    """Sử dụng cổng Sigmoid để kiểm soát luồng thông tin từ Main và các con."""
    def __init__(self, dim):
        super().__init__()
        # Input là 3 views nối tầng (Main, LC, RC) -> Output là 3 bộ trọng số tương ứng
        self.gate_layer = nn.Sequential(
            nn.Linear(dim * 3, dim * 3),
            nn.Sigmoid()
        )
        self.project = nn.Linear(dim * 3, dim)
        self.layer_norm = nn.LayerNorm(dim)

    def forward(self, views):
        # views shape: [batch, 3, dim] (đã được trích xuất cho từng token trên stack)
        # Lưu ý: Ở đây xử lý cho từng vị trí trên cửa sổ stack (S0, S1, S2)
        batch, n_tokens, n_views, dim = views.shape
        
        # Flatten views: [batch, n_tokens, dim * 3]
        flat_views = views.view(batch, n_tokens, -1)
        
        # Tính toán bộ lọc (gate)
        gates = self.gate_layer(flat_views) # [batch, n_tokens, dim * 3]
        
        # Áp dụng lọc thông tin
        gated_info = flat_views * gates
        
        # Chiếu về lại không gian ẩn node_dim
        fused = self.project(gated_info)
        return self.layer_norm(fused)

class BiaffineRelationalProjector(nn.Module):
    """Sử dụng Biaffine để trích xuất đặc trưng quan hệ Cha-Con thay cho MLP."""
    def __init__(self, head_dim, child_dim, output_dim, dropout_rate=0.2):
        super().__init__()
        # Ma trận Biaffine để học tương tác (Bilinear)
        # output_dim ở đây đóng vai trò như các 'feature maps' của quan hệ
        self.weight = nn.Parameter(torch.Tensor(output_dim, head_dim + 1, child_dim + 1))
        
        self.norm = nn.LayerNorm(output_dim)
        self.mish = nn.Mish()
        self.dropout = nn.Dropout(dropout_rate)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, head_embed, child_embed):
        # 1. Thêm bias cho cả head và child
        # head: [batch, dim] -> [batch, dim+1]
        h = torch.cat([head_embed, head_embed.new_ones(head_embed.shape[:-1]).unsqueeze(-1)], -1)
        c = torch.cat([child_embed, child_embed.new_ones(child_embed.shape[:-1]).unsqueeze(-1)], -1)
        
        # 2. Tính tương tác Biaffine: [batch, output_dim]
        # Mỗi chiều của output_dim sẽ học một loại 'khớp nối' khác nhau giữa cha và con
        rel_feat = torch.einsum('bi,oij,bj->bo', h, self.weight, c)
        
        return self.dropout(self.mish(self.norm(rel_feat)))

class BiaffineDependencyModel(nn.Module):
    def __init__(self,
                 pos_vocab_size, dep_vocab_size, num_actions,
                 phobert_dim=768, pos_dim=64, dep_dim=64,
                 node_dim=256, hidden_dim=1024, dropout_rate=0.5):
        super().__init__()

        self.node_dim = node_dim

        self.pos_embed = nn.Embedding(pos_vocab_size, pos_dim, padding_idx=0)
        self.dep_embed = nn.Embedding(dep_vocab_size, dep_dim, padding_idx=0)

        # 1. Projector cho Main Word
        self.main_proj = SubViewProjector(phobert_dim + pos_dim, node_dim * 2, node_dim)
        
        # 2. Relational Projectors cho Children (Dùng thông tin cha để hiểu con)
        # Input dim = node_dim (của cha) + (phobert + dep + pos của con)
        child_input_dim = phobert_dim + dep_dim + pos_dim
        self.lc_rel_proj = BiaffineRelationalProjector(node_dim, child_input_dim, node_dim)
        self.rc_rel_proj = BiaffineRelationalProjector(node_dim, child_input_dim, node_dim)

        self.fusion = GatedFusion(node_dim)
        self.buffer_proj = SubViewProjector(phobert_dim + pos_dim, node_dim * 2, node_dim)

        combined_dim = 4 * node_dim

        # 3. Classifier
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.Mish(), # Dùng Mish cho mượt hơn ReLU
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Mish(),
            nn.Linear(hidden_dim // 2, num_actions)
        )
        self.apply(self._init_weights)

    def forward(self, s_vecs, s_pos, s_lc_vecs, s_lc_dep, s_lc_pos, s_rc_vecs, s_rc_dep, s_rc_pos, b_vecs, b_pos):
        # --- BƯỚC 1: Chiếu nút gốc (Main Head) ---
        v_main_s = self.main_proj(torch.cat([s_vecs, self.pos_embed(s_pos)], dim=-1))

        # --- BƯỚC 2: Chiếu Relational cho nút con ---
        # Chúng ta dùng v_main_s như ngữ cảnh để "nhìn" các nút con
        feat_lc = torch.cat([s_lc_vecs, self.dep_embed(s_lc_dep), self.pos_embed(s_lc_pos)], dim=-1)
        feat_rc = torch.cat([s_rc_vecs, self.dep_embed(s_rc_dep), self.pos_embed(s_rc_pos)], dim=-1)
        
        v_lc = self.lc_rel_proj(v_main_s.view(-1, self.node_dim), feat_lc.view(-1, feat_lc.size(-1)))
        v_rc = self.rc_rel_proj(v_main_s.view(-1, self.node_dim), feat_rc.view(-1, feat_rc.size(-1)))
        
        # Đưa về shape cũ: [batch, 3, node_dim]
        v_lc = v_lc.view(v_main_s.shape)
        v_rc = v_rc.view(v_main_s.shape)

        # --- BƯỚC 3: Gộp thông tin ---
        stack_views = torch.stack([v_main_s, v_lc, v_rc], dim=2)
        s_node_emb = self.fusion(stack_views) + v_main_s 
        
        # Buffer & Feature Selection giữ nguyên logic cũ
        b_node_emb = self.buffer_proj(torch.cat([b_vecs, self.pos_embed(b_pos)], dim=-1))

        s0 = s_node_emb[:, -1, :] # Top 1 stack
        s1 = s_node_emb[:, -2, :] # Top 2 stack

        b0 = b_node_emb[:, 0, :]
        b1 = b_node_emb[:, 1, :]
  
        combined = torch.cat([s0, s1, b0, b1], dim=-1)
        return self.classifier(combined)
        
    def _init_weights(self, module):
        """Hàm này sẽ được apply đệ quy cho mọi module con"""
        if isinstance(module, nn.Linear):
            # Khởi tạo Linear: Xavier phù hợp với Mish/Tanh
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
                
        elif isinstance(module, nn.LayerNorm):
            # KHÔNG dùng Xavier cho LayerNorm. Giữ nguyên 1.0 và 0.0
            nn.init.constant_(module.weight, 1.0)
            nn.init.constant_(module.bias, 0.0)
            
        elif isinstance(module, nn.Embedding):
            # Khởi tạo Embedding mượt hơn cho tiếng Việt
            nn.init.normal_(module.weight, mean=0, std=0.02)


In [41]:
!pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_tfidf_weights(sentences):
    """
    sentences: List các list chứa token dict: [[{'form': 'Học_sinh', ...}, ...], ...]
    Trả về: Dictionary {word: weight}
    """
    # Chuyển đổi format từ list of dicts sang list of strings (cách nhau bởi khoảng trắng)
    corpus = [" ".join([token['form'] for token in sent]) for sent in sentences]
    
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b") # Giữ nguyên các từ có dấu/gạch dưới
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # Lấy giá trị TF-IDF trung bình của từng từ trên toàn bộ tập dữ liệu làm trọng số
    # (Hoặc bạn có thể lấy giá trị Max tùy vào mục đích)
    weights = tfidf_matrix.mean(axis=0).A1
    words = vectorizer.get_feature_names_out()
    
    return dict(zip(words, weights))

In [42]:
def pre_encode_sentences(sentences, model_name="vinai/phobert-base", device='cuda'):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    phobert = AutoModel.from_pretrained(model_name).to(device)
    phobert.eval()

    encoded_bank = {}
    for idx, sent in enumerate(tqdm(sentences)):
        # CHỈNH SỬA TẠI ĐÂY: Chuyển khoảng trắng thành gạch dưới để khớp với vocab PhoBERT
        words = [token['form'].replace(' ', '_') for token in sent] 

        inputs = tokenizer(words,
                           is_split_into_words=True,
                           return_tensors="pt",
                           truncation=True,
                           max_length=256).to(device)

        with torch.no_grad():
            outputs = phobert(**inputs)
            full_embeds = outputs.last_hidden_state.squeeze(0)

        word_level_embeds = []
        ptr = 1 
        for word in words:
            # Tokenizer lúc này sẽ xử lý 'Trả_lời' như một đơn vị
            sub_tokens_for_word = tokenizer.tokenize(word)
            num_subtokens = len(sub_tokens_for_word)

            word_vecs = full_embeds[ptr : ptr + num_subtokens]
            # Nếu 'Trả_lời' bị tách thành nhiều mảnh (sub-words), 
            # Mean Pooling ở đây sẽ gộp chúng lại thành 1 vector duy nhất cho từ đó
            word_mean_vec = word_vecs.mean(dim=0) if word_vecs.size(0) > 0 else torch.zeros(768, device=device)
            word_level_embeds.append(word_mean_vec)
            ptr += num_subtokens

        root_vector = full_embeds[0]
        final_matrix = torch.stack([root_vector] + word_level_embeds)
        encoded_bank[idx] = final_matrix.cpu()

    return encoded_bank

In [43]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

def pre_encode_sentences_tfidf(sentences, model_name="vinai/phobert-base", device='cuda'):
    # 1. Khởi tạo Model & Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    phobert = AutoModel.from_pretrained(model_name).to(device)
    phobert.eval()

    # 2. Tính toán TF-IDF weights trước
    print("--- Đang tính toán trọng số TF-IDF ---")
    tfidf_dict = compute_tfidf_weights(sentences)

    encoded_bank = {}
    print(f"--- Đang trích xuất TF-IDF Weighted Embedding cho {len(sentences)} câu ---")

    for idx, sent in enumerate(tqdm(sentences)):
        words = [token['form'] for token in sent]

        # Encode câu
        inputs = tokenizer(words, is_split_into_words=True, return_tensors="pt",
                           truncation=True, max_length=256).to(device)

        with torch.no_grad():
            outputs = phobert(**inputs)
            full_embeds = outputs.last_hidden_state.squeeze(0)

        word_level_embeds = []
        ptr = 1  # Bỏ qua token <s>

        for word in words:
            sub_tokens = tokenizer.tokenize(word)
            num_subtokens = len(sub_tokens)
            
            # Lấy các vector sub-tokens và tính Mean để ra vector từ gốc
            word_vecs = full_embeds[ptr : ptr + num_subtokens]
            
            if word_vecs.size(0) > 0:
                # Mean pooling cho các sub-tokens
                mean_vec = word_vecs.mean(dim=0)
                
                # Áp dụng trọng số TF-IDF (mặc định là 1.0 nếu từ không có trong TF-IDF)
                # Lưu ý: .lower() để đồng bộ hóa key
                weight = tfidf_dict.get(word.lower(), 1.0)
                
                # Để tránh trọng số quá nhỏ (gần bằng 0), bạn có thể dùng công thức: (1 + weight)
                weighted_vec = mean_vec * weight
                word_level_embeds.append(weighted_vec)
            else:
                word_level_embeds.append(torch.zeros(768, device=device))

            ptr += num_subtokens

        # Vector ROOT (<s>) - Thường giữ nguyên trọng số 1.0
        root_vector = full_embeds[0]

        final_matrix = torch.stack([root_vector] + word_level_embeds)
        encoded_bank[idx] = final_matrix.cpu()

    return encoded_bank

In [44]:
# Tạo bản đồ metadata cho hành động để tránh parse chuỗi trong vòng lặp
def build_action_metadata(id_to_action, d_vocab):
    action_meta = {}
    for act_id, name in id_to_action.items():
        if name == "SHIFT":
            action_meta[act_id] = {'type': 'S', 'label': None, 'dep_id': 0}
        elif name.startswith("LEFT-ARC"):
            label = name.split("_")[-1]
            action_meta[act_id] = {'type': 'L', 'label': label, 'dep_id': d_vocab.get(label, 0)}
        elif name.startswith("RIGHT-ARC"):
            label = name.split("_")[-1]
            action_meta[act_id] = {'type': 'R', 'label': label, 'dep_id': d_vocab.get(label, 0)}
    return action_meta

# Gọi hàm này TRƯỚC khi vào vòng lặp đánh giá (ngoài hàm process_one_sentence)
# action_meta = build_action_metadata(id_to_action, oracle.d_vocab)

In [45]:
import copy
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

def train(model, train_loader, train_encoded_bank, dev_sentences, dev_encoded_bank,
          oracle, epochs=20, lr=0.001, weight_decay=1e-4, device='cuda'):

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # --- THÊM SCHEDULER ---
    # Giảm LR đi 0.5 lần nếu UAS không tăng sau 3 epoch
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-5,
    )

    criterion = nn.CrossEntropyLoss(label_smoothing=0.05) # Giảm smoothing xuống 0.05 như đã bàn
    model.to(device)

    best_uas = 0.0
    best_model_state = None
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        batch_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for (s_words, s_pos, s_lc_idx, s_lc_dep, s_lc_pos, s_rc_idx, s_rc_dep, s_rc_pos,
             b_words, b_pos, labels, sent_ids) in batch_bar: 

            optimizer.zero_grad()

            # Helper để fetch vector từ bank
            def fetch_vecs(indices, sids):
                return torch.stack([
                    train_encoded_bank[sid][indices[i]]
                    for i, sid in enumerate(sids)
                ]).to(device)

            s_vecs    = fetch_vecs(s_words, sent_ids)
            s_lc_vecs = fetch_vecs(s_lc_idx, sent_ids)
            s_rc_vecs = fetch_vecs(s_rc_idx, sent_ids)
            b_vecs    = fetch_vecs(b_words, sent_ids)

            # Move auxiliary data to device
            s_pos, s_lc_dep, s_lc_pos, s_rc_dep, s_rc_pos, b_pos, labels = [
                x.to(device) for x in [s_pos, s_lc_dep, s_lc_pos, s_rc_dep, s_rc_pos, b_pos, labels]
            ]

            # Forward
            logits = model(s_vecs, s_pos, s_lc_vecs, s_lc_dep, s_lc_pos, s_rc_vecs, s_rc_dep, s_rc_pos, b_vecs, b_pos)

            loss = criterion(logits, labels)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            batch_bar.set_postfix(loss=f"{loss.item():.4f}", lr=f"{optimizer.param_groups[0]['lr']:.6f}")

        # --- 4. EVALUATION ---
        print(f"\n🔍 Đang đánh giá Epoch {epoch+1}...")
        # should_eval_full = (epoch + 1) % 3 == 0 or epoch > 15
        # eval_sentences = dev_sentences if should_eval_full else dev_sentences[:200] # Subset 200 câu
        uas, las = evaluate_model_batch(
            model, oracle, dev_sentences, dev_encoded_bank, device=device
        )
        print(f"✨ Kết quả: UAS: {uas*100:.2f}% | LAS: {las*100:.2f}%")

        # --- CẬP NHẬT SCHEDULER DỰA TRÊN UAS ---
        scheduler.step(uas)

        # --- 5. SAVE MODEL IF IMPROVED ---
        if uas > best_uas:
            best_uas = uas
            best_model_state = copy.deepcopy(model.state_dict())
            uas_str = f"{uas:.4f}"
            save_path = f"best_model_bi_linear_scorer_{epoch+1}_{uas_str}.pth"
            torch.save(best_model_state, save_path)
            print(f"✅ New best model saved: {save_path}")

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"🥇 Hoàn tất. UAS cao nhất: {best_uas*100:.2f}%")

    return model

In [46]:
def process_one_sentence(sentence, sent_idx, model, encoded_bank, oracle, action_meta, device):
    model.eval()
    stack, buffer, arcs = [0], list(range(1, len(sentence) + 1)), []
    left_children, right_children = {}, {}

    # Chuyển matrix sang device 1 lần duy nhất
    sent_matrix = encoded_bank[sent_idx].to(device)

    while buffer or len(stack) > 1:
        # 1. Trích xuất đặc trưng (vẫn dùng hàm extract_features đã tối ưu của bạn)
        feat = oracle.extract_features(stack, buffer, sentence, sent_idx, left_children, right_children)
        s_f, b_f = feat['stack'], feat['buffer']

        # 2. CHUẨN BỊ TENSORS (Tối ưu: Chỉ gọi .to(device) 1 lần cho mỗi nhóm)
        # Các feature ID thường nhỏ, gom nhóm để chuyển device hiệu quả hơn
        s_idx = torch.tensor(s_f['words'], dtype=torch.long, device=device)
        slc_idx = torch.tensor(s_f['l_child_idx'], dtype=torch.long, device=device)
        src_idx = torch.tensor(s_f['r_child_idx'], dtype=torch.long, device=device)
        
        # Nhặt vector PhoBERT
        s_vecs, s_lc_vecs, s_rc_vecs = sent_matrix[s_idx].unsqueeze(0), sent_matrix[slc_idx].unsqueeze(0), sent_matrix[src_idx].unsqueeze(0)

        # Labels phụ
        s_pos = torch.tensor(s_f['pos'], dtype=torch.long, device=device).unsqueeze(0)
        s_lc_dep = torch.tensor(s_f['l_child_dep'], dtype=torch.long, device=device).unsqueeze(0)
        s_rc_dep = torch.tensor(s_f['r_child_dep'], dtype=torch.long, device=device).unsqueeze(0)
        s_lc_pos = torch.tensor(s_f['l_child_pos'], dtype=torch.long, device=device).unsqueeze(0)
        s_rc_pos = torch.tensor(s_f['r_child_pos'], dtype=torch.long, device=device).unsqueeze(0)

        b_idx = torch.tensor(b_f['words'], dtype=torch.long, device=device)
        b_vecs = sent_matrix[b_idx].unsqueeze(0)
        b_pos = torch.tensor(b_f['pos'], dtype=torch.long, device=device).unsqueeze(0)

        # 3. DỰ ĐOÁN
        with torch.no_grad():
            logits = model(s_vecs, s_pos, s_lc_vecs, s_lc_dep, s_lc_pos, s_rc_vecs, s_rc_dep, s_rc_pos, b_vecs, b_pos)
            # Lấy list ID hành động đã sắp xếp
            sorted_act_ids = torch.argsort(logits[0], descending=True).tolist()

        # 4. ÁP DỤNG HÀNH ĐỘNG (Dùng action_meta để bỏ qua parse chuỗi)
        applied = False
        for act_id in sorted_act_ids:
            meta = action_meta[act_id]
            act_type = meta['type']
            
            if act_type == 'S' and buffer: # SHIFT
                stack.append(buffer.pop(0))
                applied = True
                break
                
            elif act_type == 'L' and len(stack) >= 2 and stack[-2] != 0: # LEFT-ARC
                head, child = stack[-1], stack[-2]
                arcs.append((head, child, meta['label']))
                
                # Lưu vào children map dùng ID có sẵn trong meta và sentence
                if head not in left_children: left_children[head] = []
                left_children[head].append((child, meta['dep_id'], sentence[child-1]['upos_id']))
                
                stack.pop(-2)
                applied = True
                break
                
            elif act_type == 'R' and len(stack) >= 2: # RIGHT-ARC
                head, child = stack[-2], stack[-1]
                arcs.append((head, child, meta['label']))
                
                # Lưu vào children map
                if head not in right_children: right_children[head] = []
                right_children[head].append((child, meta['dep_id'], sentence[child-1]['upos_id']))
                
                stack.pop(-1)
                applied = True
                break

        if not applied: break

    # 5. TÍNH UAS/LAS (Giữ nguyên logic chính nhưng viết gọn)
    gold_heads = {t['id']: t['head'] for t in sentence}
    gold_deps = {t['id']: t['deprel'] for t in sentence}
    pred_info = {child: (head, label) for head, child, label in arcs}

    correct_uas = sum(1 for c, g_h in gold_heads.items() if c in pred_info and pred_info[c][0] == g_h)
    correct_las = sum(1 for c, g_h in gold_heads.items() if c in pred_info and pred_info[c][0] == g_h and pred_info[c][1] == gold_deps[c])

    return correct_uas, correct_las, len(gold_heads)

In [47]:
import torch
import torch.nn.functional as F

def process_one_sentence_beam(sentence, sent_idx, model, encoded_bank, oracle, id_to_action, device, beam_size=3):
    model.eval()
    
    # Ma trận PhoBERT cho câu (Lookup table)
    sent_matrix = encoded_bank[sent_idx].to(device)
    
    # Một trạng thái bao gồm: (stack, buffer, arcs, left_children, right_children, log_score, is_finished)
    # Khởi tạo: stack=[0], buffer=1->N, arcs=[], lc={}, rc={}, score=0.0, finished=False
    initial_state = ([0], list(range(1, len(sentence) + 1)), [], {}, {}, 0.0, False)
    beam = [initial_state]

    # Số bước tối đa trong transition-based parsing (Arc-Standard/Arc-Eager) thường là 2*n
    max_steps = 2 * len(sentence)

    for _ in range(max_steps):
        candidates = []
        active_indices = []
        
        # --- 1. BATCHING: Collect features for all active candidates ---
        batch_inputs = {
            's_idx': [], 'slc_idx': [], 'src_idx': [],
            's_pos': [], 's_lc_dep': [], 's_lc_pos': [], 's_rc_dep': [], 's_rc_pos': [],
            'b_idx': [], 'b_pos': []
        }

        for i, state in enumerate(beam):
            stack, buffer, arcs, lc, rc, score, finished = state
            # Nếu trạng thái đã kết thúc hoặc không thể thực hiện thêm hành động
            if finished or (not buffer and len(stack) <= 1):
                candidates.append(state)
                continue
            
            active_indices.append(i)
            
            # --- 1. TRÍCH XUẤT ĐẶC TRƯNG TỪ TRẠNG THÁI HIỆN TẠI ---
            feat = oracle.extract_features(stack, buffer, sentence, sent_idx, lc, rc)
            
            # Collect features for batching
            s_f, b_f = feat['stack'], feat['buffer']
            batch_inputs['s_idx'].append(s_f['words'])
            batch_inputs['slc_idx'].append(s_f['l_child_idx'])
            batch_inputs['src_idx'].append(s_f['r_child_idx'])
            batch_inputs['s_pos'].append(s_f['pos'])
            batch_inputs['s_lc_dep'].append(s_f['l_child_dep'])
            batch_inputs['s_rc_dep'].append(s_f['r_child_dep'])
            batch_inputs['s_lc_pos'].append(s_f['l_child_pos'])
            batch_inputs['s_rc_pos'].append(s_f['r_child_pos'])
            batch_inputs['b_idx'].append(b_f['words'])
            batch_inputs['b_pos'].append(b_f['pos'])

        if not active_indices:
            break

        # --- 2. PREPARE BATCH TENSORS ---
        def to_tensor(data):
            return torch.tensor(data, dtype=torch.long, device=device)

        s_idx_t = to_tensor(batch_inputs['s_idx'])
        slc_idx_t = to_tensor(batch_inputs['slc_idx'])
        src_idx_t = to_tensor(batch_inputs['src_idx'])
        s_pos_t = to_tensor(batch_inputs['s_pos'])
        s_lc_dep_t = to_tensor(batch_inputs['s_lc_dep'])
        s_rc_dep_t = to_tensor(batch_inputs['s_rc_dep'])
        s_lc_pos_t = to_tensor(batch_inputs['s_lc_pos'])
        s_rc_pos_t = to_tensor(batch_inputs['s_rc_pos'])
        b_idx_t = to_tensor(batch_inputs['b_idx'])
        b_pos_t = to_tensor(batch_inputs['b_pos'])

        # Lookup embeddings (Batch lookup)
        s_vecs = sent_matrix[s_idx_t]
        s_lc_vecs = sent_matrix[slc_idx_t]
        s_rc_vecs = sent_matrix[src_idx_t]
        b_vecs = sent_matrix[b_idx_t]

        # --- 3. RUN MODEL ON BATCH ---
        with torch.no_grad():
            logits = model(
                s_vecs, s_pos_t, s_lc_vecs, s_lc_dep_t, s_lc_pos_t,
                s_rc_vecs, s_rc_dep_t, s_rc_pos_t, b_vecs, b_pos_t
            )
            log_probs = F.log_softmax(logits, dim=1)

        # --- 4. GENERATE CANDIDATES ---
        for idx_in_batch, beam_idx in enumerate(active_indices):
            stack, buffer, arcs, lc, rc, score, finished = beam[beam_idx]
            probs = log_probs[idx_in_batch]
            
            # Lấy top K hành động để giảm chi phí tính toán
            top_k = torch.topk(probs, k=min(beam_size * 2, len(probs)))
            
            for i in range(top_k.values.size(0)):
                act_id = top_k.indices[i].item()
                log_p = top_k.values[i].item()
                
                action_name = id_to_action[act_id]
                new_score = score + log_p
                
                # 1. Hành động SHIFT
                if action_name == "SHIFT" and buffer:
                    new_stack = stack + [buffer[0]]
                    new_buffer = buffer[1:]
                    candidates.append((new_stack, new_buffer, list(arcs), lc, rc, new_score, False))
                
                # 2. Hành động LEFT-ARC
                elif action_name.startswith("LEFT-ARC") and len(stack) >= 2:
                    if stack[-2] != 0: # Không để ROOT làm con
                        label = action_name.split("_")[-1]
                        dep_id = oracle.d_vocab.get(label, 0)
                        
                        # --- MỚI: Trích xuất POS ID của child (stack[-2]) ---
                        child_idx = stack[-2]
                        child_pos_str = sentence[child_idx - 1]['upos']
                        child_pos_id = oracle.p_vocab.get(child_pos_str, 0)
                        
                        new_arcs = arcs + [(stack[-1], stack[-2], label)]
                        new_stack = stack[:]
                        new_stack.pop(-2)
                        
                        # Cập nhật map con trái: thêm bộ 3 (index, dep_id, pos_id)
                        new_lc = lc.copy()
                        head = stack[-1]
                        new_lc[head] = new_lc.get(head, []) + [(child_idx, dep_id, child_pos_id)]
                        
                        candidates.append((new_stack, list(buffer), new_arcs, new_lc, rc, new_score, False))
                
                # 3. Hành động RIGHT-ARC
                elif action_name.startswith("RIGHT-ARC") and len(stack) >= 2:
                    label = action_name.split("_")[-1]
                    dep_id = oracle.d_vocab.get(label, 0)
                    
                    # --- MỚI: Trích xuất POS ID của child (stack[-1]) ---
                    child_idx = stack[-1]
                    child_pos_str = sentence[child_idx - 1]['upos']
                    child_pos_id = oracle.p_vocab.get(child_pos_str, 0)
                    
                    new_arcs = arcs + [(stack[-2], stack[-1], label)]
                    new_stack = stack[:]
                    new_stack.pop(-1)

                    # Cập nhật map con phải: thêm bộ 3 (index, dep_id, pos_id)
                    new_rc = rc.copy()
                    head = stack[-2]
                    new_rc[head] = new_rc.get(head, []) + [(child_idx, dep_id, child_pos_id)]
                    
                    candidates.append((new_stack, list(buffer), new_arcs, lc, new_rc, new_score, False))

        # --- 5. PRUNING ---
        beam = sorted(candidates, key=lambda x: x[5], reverse=True)[:beam_size]

    # --- 5. LẤY KẾT QUẢ TỐT NHẤT TỪ BEAM ---
    # State: (stack, buffer, arcs, lc, rc, score, finished)
    best_arcs = beam[0][2]

    # --- 6. TÍNH UAS/LAS ---
    gold_heads = {t['id']: t['head'] for t in sentence}
    gold_deps = {t['id']: t['deprel'] for t in sentence}
    pred_info = {child: (head, label) for head, child, label in best_arcs}

    correct_uas, correct_las = 0, 0
    for child, g_head in gold_heads.items():
        if child in pred_info:
            p_head, p_label = pred_info[child]
            if p_head == g_head:
                correct_uas += 1
                if p_label == gold_deps[child]:
                    correct_las += 1

    return correct_uas, correct_las, len(gold_heads)

In [48]:
def evaluate_model_parallel(model, oracle, sentences, encoded_bank, device='cpu', is_training=True):
    model.eval()
    model.to(device) # Chuyển model về device (cuda hoặc cpu)

    id_to_action = {v: k for k, v in oracle.action_map.items()}
    total_uas, total_las, total_deps = 0, 0, 0

    action_meta = build_action_metadata(id_to_action, oracle.d_vocab)

    # Sử dụng tqdm để thấy tiến trình
    with torch.no_grad(): # Tắt gradient để tiết kiệm bộ nhớ
        for idx, sent in enumerate(tqdm(sentences, desc="Evaluating")):
            # Gọi process_one_sentence với logic trích xuất feature mới
            if not is_training:
                c_uas, c_las, c_deps = process_one_sentence_beam(
                    sent, idx, model, encoded_bank, oracle, id_to_action, device
                )
            else:
                c_uas, c_las, c_deps = process_one_sentence(
                    sent, idx, model, encoded_bank, oracle, action_meta, device
                )
            total_uas += c_uas
            total_las += c_las
            total_deps += c_deps

    uas = total_uas / total_deps if total_deps > 0 else 0
    las = total_las / total_deps if total_deps > 0 else 0

    return uas, las

In [49]:
def process_batch_sentences(batch_sentences, batch_indices, model, encoded_bank, oracle, action_meta, device):
    model.eval()
    n_sentences = len(batch_sentences)
    
    states = []
    for sent in batch_sentences:
        states.append({
            'stack': [0],
            'buffer': list(range(1, len(sent) + 1)),
            'arcs': [],
            'lc': {},
            'rc': {},
            'finished': False
        })

    max_steps = max(len(s) for s in batch_sentences) * 2
    
    for _ in range(max_steps):
        active_indices = [i for i, s in enumerate(states) if not s['finished']]
        if not active_indices:
            break
            
        # 1. Thu thập Tensors đã trích xuất (có kích thước cố định theo window size)
        list_s_vecs, list_slc_vecs, list_src_vecs, list_b_vecs = [], [], [], []
        list_s_pos, list_slc_dep, list_slc_pos, list_src_dep, list_src_pos = [], [], [], [], []
        list_b_pos = []
        
        for i in active_indices:
            st = states[i]
            sent_idx = batch_indices[i]
            # Trích xuất đặc trưng thô (indices)
            f = oracle.extract_features(st['stack'], st['buffer'], batch_sentences[i], 
                                        sent_idx, st['lc'], st['rc'])
            
            # Lấy ma trận embedding của câu từ bank
            sent_mat = encoded_bank[sent_idx].to(device)
            
            # --- TRÍCH XUẤT VECTOR CỐ ĐỊNH TẠI ĐÂY ---
            # f['stack']['words'] luôn có kích thước cố định (mặc định là 3)
            s_idx_t = torch.tensor(f['stack']['words'], dtype=torch.long, device=device)
            slc_idx_t = torch.tensor(f['stack']['l_child_idx'], dtype=torch.long, device=device)
            src_idx_t = torch.tensor(f['stack']['r_child_idx'], dtype=torch.long, device=device)
            b_idx_t = torch.tensor(f['buffer']['words'], dtype=torch.long, device=device)

            list_s_vecs.append(sent_mat[s_idx_t])
            list_slc_vecs.append(sent_mat[slc_idx_t])
            list_src_vecs.append(sent_mat[src_idx_t])
            list_b_vecs.append(sent_mat[b_idx_t])

            # Thu thập các nhãn POS/Dep
            def to_long_t(data): return torch.tensor(data, dtype=torch.long, device=device)
            list_s_pos.append(to_long_t(f['stack']['pos']))
            list_slc_dep.append(to_long_t(f['stack']['l_child_dep']))
            list_slc_pos.append(to_long_t(f['stack']['l_child_pos']))
            list_src_dep.append(to_long_t(f['stack']['r_child_dep']))
            list_src_pos.append(to_long_t(f['stack']['r_child_pos']))
            list_b_pos.append(to_long_t(f['buffer']['pos']))

        # 2. Bây giờ có thể stack vì mọi tensor trong list đều cùng kích thước window
        s_vecs = torch.stack(list_s_vecs)
        s_lc_vecs = torch.stack(list_slc_vecs)
        s_rc_vecs = torch.stack(list_src_vecs)
        b_vecs = torch.stack(list_b_vecs)
        
        s_pos = torch.stack(list_s_pos)
        s_lc_dep = torch.stack(list_slc_dep)
        s_lc_pos = torch.stack(list_slc_pos)
        s_src_dep = torch.stack(list_src_dep)
        s_src_pos = torch.stack(list_src_pos)
        b_pos = torch.stack(list_b_pos)

        # 3. Batch Forward
        with torch.no_grad():
            logits = model(s_vecs, s_pos, s_lc_vecs, s_lc_dep, s_lc_pos, 
                           s_rc_vecs, s_src_dep, s_src_pos, b_vecs, b_pos)
            top_actions = torch.argsort(logits, dim=1, descending=True)

        # 4. Cập nhật trạng thái từng câu (Giữ nguyên logic transition)
        for idx_in_active, i in enumerate(active_indices):
            st = states[i]
            sentence = batch_sentences[i]
            applied = False
            
            for act_id in top_actions[idx_in_active].tolist():
                meta = action_meta[act_id]
                # Logic xử lý SHIFT, LEFT-ARC, RIGHT-ARC tương tự code cũ của bạn
                # ... (giữ nguyên phần áp dụng hành động) ...
                if meta['type'] == 'S' and st['buffer']:
                    st['stack'].append(st['buffer'].pop(0)); applied = True; break
                elif meta['type'] == 'L' and len(st['stack']) >= 2 and st['stack'][-2] != 0:
                    h, c = st['stack'][-1], st['stack'][-2]
                    st['arcs'].append((h, c, meta['label']))
                    if h not in st['lc']: st['lc'][h] = []
                    st['lc'][h].append((c, meta['dep_id'], sentence[c-1]['upos_id']))
                    st['stack'].pop(-2); applied = True; break
                elif meta['type'] == 'R' and len(st['stack']) >= 2:
                    h, c = st['stack'][-2], st['stack'][-1]
                    st['arcs'].append((h, c, meta['label']))
                    if h not in st['rc']: st['rc'][h] = []
                    st['rc'][h].append((c, meta['dep_id'], sentence[c-1]['upos_id']))
                    st['stack'].pop(-1); applied = True; break
            
            if not applied or (not st['buffer'] and len(st['stack']) <= 1):
                st['finished'] = True

    # 5. Tính toán kết quả UAS/LAS cuối batch
    b_uas, b_las, b_deps = 0, 0, 0
    for i in range(n_sentences):
        gold_heads = {t['id']: t['head'] for t in batch_sentences[i]}
        gold_deps = {t['id']: t['deprel'] for t in batch_sentences[i]}
        pred_info = {c: (h, l) for h, c, l in states[i]['arcs']}
        b_uas += sum(1 for c, gh in gold_heads.items() if c in pred_info and pred_info[c][0] == gh)
        b_las += sum(1 for c, gh in gold_heads.items() if c in pred_info and pred_info[c][0] == gh and pred_info[c][1] == gold_deps[c])
        b_deps += len(gold_heads)
        
    return b_uas, b_las, b_deps

In [50]:
def evaluate_model_batch(model, oracle, sentences, encoded_bank, batch_size=64, device='cuda'):
    model.eval()
    model.to(device)
    
    id_to_action = {v: k for k, v in oracle.action_map.items()}
    action_meta = build_action_metadata(id_to_action, oracle.d_vocab)
    
    total_uas, total_las, total_deps = 0, 0, 0
    
    # Chia sentences thành các batch
    for i in tqdm(range(0, len(sentences), batch_size), desc="Batch Evaluating"):
        batch_sents = sentences[i : i + batch_size]
        batch_idxs = list(range(i, i + len(batch_sents)))
        
        c_uas, c_las, c_deps = process_batch_sentences(
            batch_sents, batch_idxs, model, encoded_bank, oracle, action_meta, device
        )
        
        total_uas += c_uas
        total_las += c_las
        total_deps += c_deps

    uas = total_uas / total_deps if total_deps > 0 else 0
    las = total_las / total_deps if total_deps > 0 else 0
    return uas, las

In [51]:
# 1. Khai báo đường dẫn và thiết bị
train_path = "UD_Vietnamese-VTB/vi_vtb-ud-train.conllu"
dev_path = "UD_Vietnamese-VTB/vi_vtb-ud-dev.conllu"
test_path = "UD_Vietnamese-VTB/vi_vtb-ud-test.conllu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Đang đọc dữ liệu và xây dựng Vocab...")
# Lưu ý: Thêm word_vocab để quản lý index cho các node con nếu cần
# Trong cell thực hiện load dữ liệu (tương tự Cell 16 trong notebook)
train_sentences, word_vocab, p_vocab, d_vocab = load_data_and_build_vocab([train_path], is_train=True)

# GỌI HÀM NÀY NGAY TẠI ĐÂY
train_sentences = map_sentence_to_ids(train_sentences, p_vocab, d_vocab)
dev_sentences = map_sentence_to_ids(load_data_and_build_vocab([dev_path], is_train=False), p_vocab, d_vocab)
test_sentences = map_sentence_to_ids(load_data_and_build_vocab([test_path], is_train=False), p_vocab, d_vocab)

vocabs = (p_vocab, d_vocab)

# 3. Khởi tạo Oracle
# Oracle cần chứa hàm create_training_data_multiview sử dụng hàm extract_features mới của chúng ta
oracle = Oracle(p_vocab, d_vocab)

Đang đọc dữ liệu và xây dựng Vocab...


In [52]:
# 4. Tạo dữ liệu huấn luyện
print(f"Đang tạo chuỗi transitions cho {len(train_sentences)} câu...")
training_samples = []
for idx, sentence in enumerate(tqdm(train_sentences)):
    # Hàm này sẽ giả lập các bước đi (Shift, Arc) và gọi extract_features tại mỗi bước
    examples = oracle.create_training_data_multiview(sentence, idx)
    training_samples.extend(examples)

# 6. Xây dựng ngân hàng Embedding PhoBERT (Mở comment)
# Bank này lưu ma trận [sent_len + 1, 768] cho từng câu
print("--- Đang trích xuất PhoBERT Embedding Bank ---")
train_encoded_bank = pre_encode_sentences(train_sentences, device=device)
dev_encoded_bank = pre_encode_sentences(dev_sentences, device=device)
test_encoded_bank = pre_encode_sentences(test_sentences, device=device)

# 7. Cấu hình DataLoader (Mở comment và cập nhật)
train_dataset = TransitionDataset(training_samples)
train_loader = DataLoader(
    train_dataset,
    batch_size=256, # Giảm nhẹ batch size nếu VRAM bị tràn do feature quá nhiều
    shuffle=True,
    collate_fn=multiview_collate_fn # Phải là hàm trả về 11 thành phần
)

Đang tạo chuỗi transitions cho 1400 câu...


  0%|          | 0/1400 [00:00<?, ?it/s]

--- Đang trích xuất PhoBERT Embedding Bank ---


  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1123 [00:00<?, ?it/s]

  0%|          | 0/800 [00:00<?, ?it/s]

In [53]:
print(len(training_samples))

48561


In [ ]:
# 8. Khởi tạo Mô hình (BỔ SUNG)
num_actions = len(oracle.action_map)
model = BiaffineDependencyModel(
    pos_vocab_size=len(p_vocab),
    dep_vocab_size=len(d_vocab),
    num_actions=num_actions,
    phobert_dim=768,
    pos_dim=64,
    dep_dim=64,
    node_dim=256,
    hidden_dim=1024,
    dropout_rate=0.3
).to(device)

print(f"✅ Model đã sẵn sàng với {num_actions} hành động.")

# 9. Bắt đầu huấn luyện
model = train(
    model=model,
    train_loader=train_loader,
    train_encoded_bank=train_encoded_bank,
    dev_sentences=dev_sentences,
    dev_encoded_bank=dev_encoded_bank,
    oracle=oracle,
    epochs=50,
    lr=3e-4,
    device=device
)

# 3. Đánh giá cuối cùng trên tập TEST
print("\n--- Đánh giá cuối cùng trên tập TEST ---")
# Tải lại trọng số tốt nhất trước khi test
uas, las = evaluate_model_parallel(
    model, oracle, test_sentences, test_encoded_bank, device=device, is_training=False
)
print(f"Final Result: UAS: {uas*100:.2f}% | LAS: {las*100:.2f}%")


✅ Model đã sẵn sàng với 159 hành động.


Epoch 1/50:   0%|          | 0/190 [00:00<?, ?it/s]


🔍 Đang đánh giá Epoch 1...


Batch Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

✨ Kết quả: UAS: 68.35% | LAS: 51.96%
✅ New best model saved: best_model_bi_linear_scorer_1_0.6835.pth


...

Batch Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

✨ Kết quả: UAS: 74.33% | LAS: 60.21%


Epoch 6/50:   0%|          | 0/190 [00:00<?, ?it/s]


🔍 Đang đánh giá Epoch 6...


Batch Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

In [33]:
model_save = BiaffineDependencyModel(
    pos_vocab_size=len(p_vocab),
    dep_vocab_size=len(d_vocab),
    num_actions=num_actions,
    phobert_dim=768,
    pos_dim=64,
    dep_dim=64,
    node_dim=256,
    hidden_dim=1024,
    dropout_rate=0.3
).to(device)

# 3. Nạp "trọng số" (weights) vào khung đã tạo
weights_path = 'best_model_biaffine/best_model_bi_linear_scorer_38_0.7754.pth'
state_dict = torch.load(weights_path, map_location=device)

# Nếu file .pth của bạn chứa trực tiếp state_dict:
model_save.load_state_dict(state_dict)

# 4. Chuyển sang chế độ đánh giá
model_save.eval()

print("✅ Đã load weights thành công!")

✅ Đã load weights thành công!


In [34]:
uas, las = evaluate_model_parallel(
    model_save, oracle, test_sentences, test_encoded_bank, device=device, is_training=False
)
print(f"Final Result: UAS: {uas*100:.2f}% | LAS: {las*100:.2f}%")

Evaluating:   0%|          | 0/800 [00:00<?, ?it/s]

Final Result: UAS: 80.68% | LAS: 68.92%
